In [9]:

import google.generativeai as genai
from PIL import Image
from dotenv import load_dotenv
import json
import os
from io import BytesIO
import pprint

In [10]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [11]:
# Gemini prompt
GEMINI_PROMPT = (
    f"The person's actual height is 170 cm. "
        "Using this height as a scaling reference, estimate the person's body measurements from the image with the highest possible accuracy. "
        "First, detect the person's gender (male or female). "
        "Analyze the image carefully, considering body shape, proportions, and posture. "
        "Identify and measure the following body parts: chest, waist, hips, inseam, thigh, neck, and arm length. "
        "Provide all measurements in centimeters, scaled proportionally to the given height. "
        "Estimate an overall accuracy score (as a percentage) based on the image quality and measurement confidence. "
        "Recommend the best clothing size (S, M, L, XL) based on the estimated measurements and gender. "
        "Return only the result as a strictly valid JSON object, formatted exactly as follows:\n\n"
        "{\n"
        f'  "height": 170,\n'
        '  "gender": "<male_or_female>",\n'
        '  "chest": <value>,\n'
        '  "waist": <value>,\n'
        '  "hips": <value>,\n'
        '  "inseam": <value>,\n'
        '  "thigh": <value>,\n'
        '  "neck": <value>,\n'
        '  "arm_length": <value>,\n'
        '  "accuracy": <percentage>,\n'
        '  "recommended_size": "<S_or_M_or_L_or_XL>"\n'
        "}\n\n"
        "Only return the JSON — no explanation, no commentary, no extra text."
)

In [12]:
def send_to_gemini_validation(image_path: str) -> dict:
    try:
        image = Image.open(image_path)

        model = genai.GenerativeModel(model_name="gemini-2.0-flash")

        response = model.generate_content(
            [GEMINI_PROMPT.strip(), image],
            generation_config={"temperature": 0.4}
        )

        text = response.text.strip()
        json_start = text.find('{')
        json_end = text.rfind('}') + 1
        json_str = text[json_start:json_end]

        return json.loads(json_str)

    except Exception as e:
        return {"valid": False, "reason": f"Gemini validation failed: {str(e)}"}

In [13]:
if __name__ == "__main__":
    image_path = "../images/front-image.png" 
    result = send_to_gemini_validation(image_path)
    pprint.pprint(result)

{'accuracy': 85,
 'arm_length': 60,
 'chest': 95,
 'gender': 'male',
 'height': 170,
 'hips': 93,
 'inseam': 78,
 'neck': 38,
 'recommended_size': 'M',
 'thigh': 55,
 'waist': 80}
